In [4]:
# DAANISH ARA YAKTA
# PUSHPENDER SHARMA
# AARYAMAN YADAV

In [5]:
!pip install pyspark
!pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9159efafba6e95ce7b05647d12ccd9e18e169921e11dd93103558f60f0d61487
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
!wikiextractor /content/enwiki-latest-pages-articles.xml.bz2

In [6]:
from operator import add
from pyspark import SparkContext, SparkConf

aaryaman_conf = SparkConf()\
        .setAppName("Wiki_Search_Engine")\
        .setMaster("local[*]")\
        .set("spark.driver.memory", "10g")\
        .set("spark.driver.maxResultSize", "4g")

pushpender_sc = SparkContext(conf=aaryaman_conf)


In [ ]:
rawData = sc.textFile(r"wiki\artile_per_line.txt")
flatten_words = rawData.flatMap(lambda x: x.split('\t')[1].split())
words_joint = flatten_words.map(lambda x: (x, 1)).reduceByKey(add)
print "Number of distinct words in the whole Wikipedia: ", words_joint.count()
# Number of distinct words in the whole Wikipedia:  26764007

In [ ]:
import re
from nltk.corpus import stopwords
from spacy.en import English
import nltk
import spacy.attrs
import time
import codecs

fo = open(r'wiki\titles.txt')
raw_text = fo.read()
fo.close()

wiki_title_words = set(raw_text.split())
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
stop_words = set(stopwords.words("english"))

NLP = English()

fo = codecs.open(r'wiki\artile_per_line.txt', 'r', encoding='utf-8') # title \t document
out = codecs.open(r"Wiki\article_lemma.txt", 'w', encoding='utf-8')


for doc in NLP.pipe(fo, n_threads=4):    
    title_words = []
    passed_title = False
    for candidate in doc:
        if '\t' not in candidate.lemma_: # title \t document
            if passed_title:
                if (candidate.lemma_ not in stop_words) and (candidate.pos_ != u'PUNCT') and \
                    (candidate.lemma_ in english_vocab or candidate.lemma_ in wiki_title_words):

                    out.write(candidate.lemma_ + u' ')
            else:
                title_words.append(candidate.orth_)
        else:
            out.write(u''.join(title_words) + candidate.orth_)
            passed_title = True

    out.write('\n')

fo.close()
out.close()

Now let's run the word count in Spark and see how many words we have left now:

In [ ]:
from operator import add
from pyspark import SparkContext, SparkConf

conf = SparkConf()\
        .setAppName("Wiki_Search_Engine")\
        .setMaster("local[*]")\
        .set("spark.driver.memory", "10g")\
        .set("spark.driver.maxResultSize", "4g")

sc = SparkContext(conf=conf)
rawData = sc.textFile(r"wiki\article_lemma.txt")
flatten_words = rawData.flatMap(lambda x: x.split('\t')[1].split())
words_joint = flatten_words.map(lambda x: (x, 1)).reduceByKey(add)
print "Number of distinct lemmas in the whole Wikipedia: ", words_joint.count()
# Number of distinct lemmas in the whole Wikipedia:  561354

Number of distinct words left in the documents after pre-processing is 561354. That means more than 98% of the words from the original documents are all removed. This means we can comfortably hash the remaining words to build our tf-idf model.

In [ ]:
sc = SparkContext(conf=conf)
rawData = sc.textFile(r"wiki\articles_lemma.txt")
documents = rawData.map(lambda line : line.split('\t')[1].split())
titles = rawData.map(lambda line : line.split('\t')[0])
titles.cache()

hashingTF = HashingTF(20000000)  #20 Million hash buckets just to make sure it fits in memory
tf = hashingTF.transform(documents)
idf = IDF(minDocFreq=10).fit(tf)
tfidf = idf.transform(tf)
tfidf.cache()

Now that we have tf-idf model for all documents. We can send queries and return the documents that have the highest score for that perticular word:

In [ ]:
QueryTF = hashingTF.transform(["tabriz"])
QueryHashValue = QueryTF.indices[0]
QueryRelevance = tfidf.map(lambda x: x[QueryHashValue])
zippedResults = QueryRelevance.zip(titles)
print "Top 10 related documents:"
for (k, v) in zippedResults.sortByKey(ascending=False).take(10):
    print v

Top 10 related documents:
Tabriz
Azerbaijan(Iran)
Safina-yiTabriz
Tabrizrug
OldAzerilanguage
TabrizKhanate
Persiancarpet
Safavidart
MachineSaziTabrizF.C.
EastAzerbaijanProvince


As you can see it works! Tabriz is a city in Iran and all the returned results are related to the word with the first link to the Wikipedia page of the city. I run this on my local machine which takes a minute or two for each query but if we distribute this on a cluster it should return the results much faster. 